# Notebook Submission 1 Kelas MLOps: Analisis Sentimen Pada Masuknya Patrick Kluivert Sebagai Pelatih Timnas Indonesia

## Import Library

In [1]:
import tensorflow as tf
import subprocess
import os
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2 
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow_transform as tft
from tfx.components.trainer.fn_args_utils import FnArgs

from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 

import tensorflow_model_analysis as tfma 
from tfx.components import Evaluator, Pusher 

from absl import logging
logging.set_verbosity(logging.INFO)

# Set Variable

In [2]:
# Variabel-variabel pipeline
PIPELINE_NAME = "sentiment-pipeline"
SCHEMA_PIPELINE_NAME = "sentiment-schema"

# Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path SQLite untuk MLMD Storage
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Direktori output untuk model
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Direktori Data
PROCESSED_DATA_PATH = "data/youtube-comments/processed_comments.csv"
INGESTION_DATA_PATH = "data/youtube-comments"

# Direktori Root Modules
ROOT_MODULES = 'modules'

# Data Preprocessing
Menjalankan proses-proses data processing yang dilakukan pada kolom comment, serta membentuk *polarity*.

In [3]:
# Jalankan Preprocessing sebelum pipeline
print("🔄 Menjalankan preprocessing data...")

try:
    result = subprocess.run(
        ["python", f"{ROOT_MODULES}/sentiment_preprocess.py"], 
        check=True, 
        capture_output=True, 
        text=True
    )
    
    print("✅ Preprocessing berhasil dijalankan!")
    print("📜 Output:")
    print(result.stdout)  # Menampilkan output dari script

except subprocess.CalledProcessError as e:
    print(f"❌ Error saat menjalankan preprocessing: {e}")
    print("📜 Log Error:")
    print(e.stderr)  # Menampilkan pesan error dari script
    raise

# Pastikan file hasil preprocessing tersedia sebelum melanjutkan pipeline
if not os.path.exists(PROCESSED_DATA_PATH):
    raise FileNotFoundError(f"❌ File hasil preprocessing tidak ditemukan: {PROCESSED_DATA_PATH}")

print(f"✅ File hasil preprocessing tersedia: {PROCESSED_DATA_PATH}")

🔄 Menjalankan preprocessing data...
✅ Preprocessing berhasil dijalankan!
📜 Output:
✅ Data tersimpan di d:\belajar data\Dicoding\MLOps\Submission\faishal_ali-pipeline\data\youtube-comments\processed_comments.csv

✅ File hasil preprocessing tersedia: data/youtube-comments/processed_comments.csv


Menampilkan Pipeline yang sudah dibentuk

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Data Ingestion

Memisahkan data menjadi train dan evaluation

In [5]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base=INGESTION_DATA_PATH, output_config=output)

In [6]:
interactive_context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data data/youtube-comments\* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 5
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Data Validation

## Summary Statistics

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to pipelines\sentiment-pipeline\StatisticsGen\statistics\6\Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to pipelines\sentiment-pipeline\StatisticsGen\statistics\6\Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 6
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Menampilkan informasi statistik dasar dari dataset yang digunakan. Statistik yang ditampilkan sudah dalam bentuk splitted data *train* dan *eval*.

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

# Make Data Schema

In [9]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
interactive_context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to pipelines\sentiment-pipeline\SchemaGen\schema\7\schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 7
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Menampilkan hasil skema data, dengan menunjukkan fitur-fitur yang ada.

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'comment',BYTES,required,,-
'polarity',INT,required,,-
